In [10]:
import numpy as np
import pandas as pd
import pickle
import os 

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import GridSearchCV
from flexible_linear import FlexibleLinearRegression

from tv_utils import CustomTVReg, integration_mesh_to_tria, get_meshes_coord_tria, get_nodes_attribute
from data_load import load_meshes_coor_tria

from joblib import Parallel, delayed

path_data = "/cobrain/groups/ml_group/data/HCP/cleaned_data/"

with open(path_data + "mean_mesh_labels", 'rb') as f:
    mean_labels = pickle.load(f)


with open(path_data + "subjects_log_jac", 'rb') as f:
    log_jac = pickle.load(f)

with open(path_data + "subjects_thinkness", 'rb') as f:
    thinkness = pickle.load(f)

with open(path_data + "subjects_mesh_area", 'rb') as f:
    mesh_area = pickle.load(f)

path_tria= "/cobrain/groups/ml_group/data/HCP/HCP/"
coord, tria = load_meshes_coor_tria(path_tria)

del coord

targets_name = ['clustering', 'rich_club', 'betweenness', 'closeness',  'degree_centrality', 'eigenvector']

targets_data = []
for name in targets_name:
    with open(path_data + name, 'rb') as f:
        targets_data  += [pickle.load(f)]

print(np.array(targets_data).shape)


params = {'C':np.linspace(1e-10, 0.1, 5),
         'cost_func': ['l2', 'l1', 'japanese']}


path_res = '/home/ayagoz/connec/results/tv/'
if not os.path.exists(path_res + 'node_to_node'):
    os.mkdir(path_res + 'node_to_node')
path_res += 'node_to_node/'

thinkness = thinkness.reshape(789,-1)
log_jac = log_jac.reshape(789,-1)
mesh_area = mesh_area.reshape(789,-1)

idx_nodes = list(range(1,4)) + list(range(5,39)) + list(range(40,71))
idx_nodes = np.array(idx_nodes)
print(idx_nodes.shape)
X = []
triangles = []
for i in range(68):
    node1 = idx_nodes[i]
    node1_tria = get_meshes_coord_tria(tria, mean_labels, i)
    print(np.array(node1_tria).shape)
    triangles += [node1_tria] 
    T = integration_mesh_to_tria(node1_tria, thinkness)
    L = integration_mesh_to_tria(node1_tria, log_jac)
    M = integration_mesh_to_tria(node1_tria, mesh_area)
    X += [[T,L,M]]
    
print('I am here')


name = ['think/', 'log_jac/', 'mesh_area/']
params = {'C':[3*1e-10, 1e-8, 1e-5, 1e-3, 0.1, 1, 10],
         'cost_func': ['l2','l1', 'japanese']}


def main_worker(i):
    node1 = idx_nodes[i]
    print(node1)
    custom_tvl1 = CustomTVReg(data = triangles[i], mode_reg= 'l1')
    custom_tvl2 = CustomTVReg(data = triangles[i], mode_reg= 'l2')

    flexl1 = FlexibleLinearRegression(C=1e-8, reg_cost_func=custom_tvl1.tv_normed_cost_func)
    flexl2 = FlexibleLinearRegression(C=1e-8, reg_cost_func=custom_tvl2.tv_normed_cost_func)
    grl1 = GridSearchCV(flexl1, param_grid=params, n_jobs=1, scoring='r2', cv = 3)
    grl2 = GridSearchCV(flexl1, param_grid=params, n_jobs=1, scoring='r2', cv = 3)

    for folder in name:
        if not os.path.exists(path_res + folder):
            os.mkdir(path_res + folder)

    print(X[i][0].shape, target[:,i].shape)



    for k in range(3):

        grl1.fit(X[i][k], target[:,i])
        grl2.fit(X[i][k], target[:,i])
        res_l1 = pd.DataFrame.from_dict(grl1.cv_results_)
        res_l1 = res_l1.sort_values(by = 'rank_test_score').iloc[:1]
        res_l1.to_csv(path_res + name[j] + 'tv_l1')

        res_l2 = pd.DataFrame.from_dict(grl2.cv_results_)
        res_l2 = res_l2.sort_values(by = 'rank_test_score').iloc[:1]
        res_l2.to_csv(path_res + name[j] + 'tv_l2')
    



for j, target in enumerate(targets_data):
    print(targets_name[j])
    print(target.shape)
    Parallel(n_jobs=1)(delayed(main_worker)(i) for i in range(68))
    print('Finished work for ' + targets_name[j])
print('I finished all')

Meshes coordinates shape:  (327684, 3)
Number of triangles of meshes:  (655360, 3)
(6, 789, 68)
(68,)
(49765, 3)
I am here


In [3]:
import numpy as np
import pandas as pd
import pickle
import os 

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import GridSearchCV
from flexible_linear import FlexibleLinearRegression

from tv_utils import CustomTVReg, integration_mesh_to_tria, get_meshes_coord_tria, get_nodes_attribute
from data_load import load_meshes_coor_tria

path_data = "/cobrain/groups/ml_group/data/HCP/cleaned_data/"

with open(path_data + "mean_mesh_labels", 'rb') as f:
    mean_labels = pickle.load(f)


with open(path_data + "subjects_log_jac", 'rb') as f:
    log_jac = pickle.load(f)

with open(path_data + "subjects_thinkness", 'rb') as f:
    thinkness = pickle.load(f)

with open(path_data + "subjects_mesh_area", 'rb') as f:
    mesh_area = pickle.load(f)

path_tria= "/cobrain/groups/ml_group/data/HCP/HCP/"
coord, tria = load_meshes_coor_tria(path_tria)

del coord

targets_name = ['clustering', 'rich_club', 'betweenness', 'closeness',  'degree_centrality', 'eigenvector']

targets_data = []
for name in targets_name:
    with open(path_data + name, 'rb') as f:
        targets_data  += [pickle.load(f)]

print(np.array(targets_data).shape)


params = {'C':np.linspace(1e-10, 0.1, 5),
         'cost_func': ['l2', 'l1', 'japanese']}


path_res = '/home/ayagoz/connec/results/tv/'
if not os.path.exists(path_res + 'node_to_node'):
    os.mkdir(path_res + 'node_to_node')
path_res += 'node_to_node/'

thinkness = thinkness.reshape(789,-1)
log_jac = log_jac.reshape(789,-1)
mesh_area = mesh_area.reshape(789,-1)

idx_nodes = list(range(1,4)) + list(range(5,39)) + list(range(40,71))
idx_nodes = np.array(idx_nodes)
print(idx_nodes.shape)
X = []
triangles = []
for i in range(68):
    node1 = idx_nodes[i]
    node1_tria = get_meshes_coord_tria(tria, mean_labels, i)
    print(np.array(node1_tria).shape)
    triangles += [node1_tria] 
    T = integration_mesh_to_tria(node1_tria, thinkness)
    L = integration_mesh_to_tria(node1_tria, log_jac)
    M = integration_mesh_to_tria(node1_tria, mesh_area)
    X += [[T,L,M]]
    break
print('I am here')


name = ['think/', 'log_jac/', 'mesh_area/']
params = {'C':[1e-8],
         'cost_func': ['l2',]}
for j, target in enumerate(targets_data):
    print(targets_name[j])
    print(target.shape)
    for i in range(68):
        
        
        node1 = idx_nodes[i]
        print(node1)
        custom_tvl1 = CustomTVReg(data = triangles[i], mode_reg= 'l1')#triangles[i]
        custom_tvl2 = CustomTVReg(data = triangles[i], mode_reg= 'l2')
        
        flexl1 = FlexibleLinearRegression(C=1e-8, reg_cost_func=custom_tvl1.tv_normed_cost_func)
        flexl2 = FlexibleLinearRegression(C=1e-8, reg_cost_func=custom_tvl2.tv_normed_cost_func, n_)
        grl1 = GridSearchCV(flexl1, param_grid=params, scoring='r2', cv = 3)
        grl2 = GridSearchCV(flexl1, param_grid=params, scoring='r2', cv = 3)
        
        for folder in name:
            if not os.path.exists(path_res + folder):
                os.mkdir(path_res + folder)
        
        
        
        
        
        for k in range(3):
            
            grl1.fit(X[i][k], target[:,i])
            
            res_think = pd.DataFrame.from_dict(grl1.cv_results_)
            res_think = res_think.sort_values(by = 'rank_test_score').iloc[:1]
            res_think.to_csv(path_res + name[j] + 'tv_l1')
        
        for k in range(3):
            grl2.fit(X[i][k], target[:,i])
            res_think = pd.DataFrame.from_dict(grl1.cv_results_)
            res_think = res_think.sort_values(by = 'rank_test_score').iloc[:1]
            res_think.to_csv(path_res + name[j] + 'tv_l2')
    print('Finished work for ' + targets_name[j])

Meshes coordinates shape:  (327684, 3)
Number of triangles of meshes:  (655360, 3)
(6, 789, 68)
(68,)
(49765, 3)
I am here
